In [1]:
# Melakukan impor libraries yang diperlukan untuk ekstraksi informasi wajah pada training images.
import numpy as np
import pandas as pd
import matplotlib.image as mpimg
import math
import cv2
import time
import datetime

In [2]:
# Melakukan impor MTCNN untuk deteksi wajah.
! pip install mtcnn
from mtcnn.mtcnn import MTCNN

     |████████████████████████████████| 2.3 MB 4.2 MB/s 


In [3]:
# Mencatat waktu dimulainya keseluruhan program ekstraksi informasi wajah pada training images.
global_start_time = time.time()

In [4]:
# Membaca sekaligus menampilkan isi dari data training.
df_train = pd.read_csv('../input/bdc-satriadata2021/train.csv')
df_train

,nomor,jenis kelamin,usia
0,1,0,27
1,2,1,24
2,3,0,29
3,4,1,23
4,5,0,20
...,...,...,...
765,766,0,21
766,767,0,25
767,768,0,21
768,769,0,23


In [5]:
# Mencatat waktu dimulainya pembacaan training images.
start_time = time.time()

# Mencatat index dari setiap training images.
img_train = {}
index_train = list(df_train.iloc[:,0])
subindex_train = [1, 2, 3]
filename_master_train = '../input/bdc-satriadata2021/Training/'

for index in index_train:
    # Membaca dan menyimpan training images.
    img_train[index] = {}
    img_index = str(index) + '/' + str(index) + '_'
    for subindex in subindex_train:
        img_subindex = str(subindex) + '.jpg'
        filename_train = filename_master_train + img_index + img_subindex
        img_train[index][subindex] = mpimg.imread(filename_train)

# Mencatat waktu berakhirnya pembacaan training images.
end_time = time.time()

# Menampilkan waktu eksekusi dari pembacaan training images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

execution time: 0:01:27.183479


In [6]:
# Mendeklarasikan pendeteksi wajah.
detector = MTCNN()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

In [7]:
# Membaca sekaligus menampilkan informasi wajah pada data training.
info_train = pd.read_csv('../input/bdc-sd2021-data-tambahan/train_facial_data.csv')
info_train = info_train[info_train.iloc[:, 2] > -1].reset_index().drop('index', axis = 1)
info_train

,nomor,urutan_gambar,x_coor,y_coor,width,height,x_lefteye,y_lefteye,x_righteye,y_righteye,x_nose,y_nose,x_mouthleft,y_mouthleft,x_mouthright,y_mouthright
0,1,1,263,176,315,431,329,357,475,348,393,448,337,494,491,484
1,1,2,460,161,229,304,499,295,588,283,531,355,517,393,611,381
2,1,3,455,701,153,203,489,789,557,777,524,826,504,855,569,841
3,2,1,592,489,108,138,622,541,671,547,639,567,614,585,667,591
4,2,2,472,407,182,252,517,510,599,509,549,557,511,587,601,591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,769,2,323,157,297,315,395,322,490,265,456,370,467,429,544,380
2301,769,3,382,504,360,422,486,724,619,642,602,780,572,854,689,787
2302,770,1,422,163,379,563,527,385,710,415,593,518,515,591,654,612
2303,770,2,248,180,296,403,303,374,411,321,358,424,369,506,451,467


In [8]:
# Menentukan ukuran gambar dan margin gambar saat akan dirotasi.
adj_ratio = 0.25
pict_size = (128, 128)

In [9]:
# Mencatat waktu dimulainya pencatatan informasi training images.
start_time = time.time()

# Menyiapkan ukuran penyimpanan informasi gambar.
nrows = info_train.shape[0]

# Menyiapkan dataframe untuk menampung informasi gambar.
facial_relative = pd.DataFrame()

for row in range(nrows):
    # Menyiapkan indeks gambar.
    nomor = info_train.iloc[row, 0]
    urutan_gambar = info_train.iloc[row, 1]
    
    # Menyiapkan gambar untuk dirotasi.
    my_image = img_train[nomor][urutan_gambar].copy()
    x_coor, y_coor, width, height = info_train.iloc[row, 2:6]
    adj_x_coor = max(x_coor-int(adj_ratio*width), 0)
    adj_y_coor = max(y_coor-int(adj_ratio*height), 0)
    adj_width = int(width*(1+2*adj_ratio))
    adj_height = int(height*(1+2*adj_ratio))
    crop = my_image[adj_y_coor:adj_y_coor+adj_height, adj_x_coor:adj_x_coor+adj_width]
    
    # Melakukan rotasi.
    delta_x_eyes = int(info_train.iloc[row, 8] - info_train.iloc[row, 6])
    delta_y_eyes = int(info_train.iloc[row, 9] - info_train.iloc[row, 7])
    x_center = crop.shape[1] // 2
    y_center = crop.shape[0] // 2
    angle = math.degrees(math.atan(delta_y_eyes/delta_x_eyes))
    M = cv2.getRotationMatrix2D((x_center, y_center), angle, 1)
    out = cv2.warpAffine(crop, M, (crop.shape[1], crop.shape[0]))
    
    # Menentukan lokasi wajah yang baru.
    face_location = detector.detect_faces(out)
    if len(face_location):
        new_x, new_y, new_w, new_h = face_location[0]['box']
        main_face = out[new_y:new_y+new_h, new_x:new_x+new_w]
        
        # Memasukkan informasi posisi relatif fitur pada wajah.
        temp_face = []
        main_element = face_location[0]['keypoints']
        for face_element in main_element:
            temp_face_0 = (np.array(main_element[face_element]) - (new_x, new_y)) / (new_w, new_h)
            temp_face.append(list(temp_face_0))
        temp_face = list(np.array(temp_face).reshape(10))
        temp_face.append(new_h / new_w)
        
    else:
        # Definisi ulang gambar, jika deteksi wajah gagal dilakukan.
        crop = my_image[y_coor:y_coor+height, x_coor:x_coor+width]
        # Melakukan rotasi.
        delta_x_eyes = int(info_train.iloc[row, 8] - info_train.iloc[row, 6])
        delta_y_eyes = int(info_train.iloc[row, 9] - info_train.iloc[row, 7])
        x_center = crop.shape[1] // 2
        y_center = crop.shape[0] // 2
        angle = math.degrees(math.atan(delta_y_eyes/delta_x_eyes))
        M = cv2.getRotationMatrix2D((x_center, y_center), angle, 1)
        out = cv2.warpAffine(crop, M, (crop.shape[1], crop.shape[0]))
        main_face = out
        
        # Memasukkan informasi posisi relatif fitur pada wajah.
        temp_face = []
        for temp_index in range(5):
            temp_face_0 = (np.array(info_train.iloc[row, 6+2*temp_index:8+2*temp_index]) - (x_coor, y_coor)) / (width, height)
            temp_face.append(list(temp_face_0))
        temp_face = list(np.array(temp_face).reshape(10))
        temp_face.append(height / width)
    
    # Memasukkan informasi posisi relatif yang diukur ulang tanpa mengubah rasio ukuran wajah.
    adj_y = []
    for temp_index in range(len(temp_face)):
        if temp_index%2:
            adj_y.append(temp_face[temp_index] * temp_face[-1])
    temp_face += adj_y
    
    # Membuat nama kolom untuk informasi wajah.
    facial_element = ['_le', '_re', '_n', '_ml', '_mr']
    col_name = []
    for temp_element in facial_element:
        col_name.append('x' + temp_element)
        col_name.append('y' + temp_element)
    col_name.append('wh_ratio')
    for temp_element in facial_element:
        col_name.append('adj_y' + temp_element)
    
    # Mencatat koordinat relatif wajah.
    x_face = []
    y_face = []
    for temp_index in range(5):
        x_face.append(temp_face[2*temp_index])
        y_face.append(temp_face[2*temp_index+1])
    
    # Memasukkan informasi jarak antar titik penting pada wajah.
    for temp_index_0 in range(5):
        for temp_index_1 in range(temp_index_0):
            temp_name = facial_element[temp_index_1] + facial_element[temp_index_0]
            delta_x = x_face[temp_index_0] - x_face[temp_index_1]
            delta_y = y_face[temp_index_0] - y_face[temp_index_1]
            adj_delta_y = delta_y * temp_face[10]
            
            # Jarak simetris.
            sym_distance = np.sqrt(delta_x**2 + delta_y**2)
            temp_face.append(sym_distance)
            col_name.append('sym' + temp_name)
            
            # Jarak yang disesuaikan dengan rasio ukuran wajah.
            adj_distance = np.sqrt(delta_x**2 + adj_delta_y**2)
            temp_face.append(adj_distance)
            col_name.append('adj' + temp_name)
    
    # Memasukkan informasi kemiringan sudut wajah pada gambar.
    temp_face.append(abs(math.radians(angle)))
    col_name.append('abs_angle')
    
    # Mengkompilasi seluruh informasi wajah dalam suatu dataframe.
    face_df = pd.DataFrame(temp_face).T
    facial_relative = pd.concat([facial_relative, face_df], ignore_index = True)

# Menyimpan seluruh informasi gambar ke dalam format .csv.
facial_relative.columns = col_name
facial_relative.to_csv('train_facial_relative.csv', index = False)

# Mencatat waktu berakhirnya pencatatan informasi training images.
end_time = time.time()

# Menampilkan waktu eksekusi dari pencatatan informasi training images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

2021-11-01 18:40:30.540254: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


execution time: 0:26:57.533298


In [10]:
# Menampilkan informasi gambar yang sudah diperoleh.
facial_relative

,x_le,y_le,x_re,y_re,x_n,y_n,x_ml,y_ml,x_mr,y_mr,...,adj_n_ml,sym_le_mr,adj_le_mr,sym_re_mr,adj_re_mr,sym_n_mr,adj_n_mr,sym_ml_mr,adj_ml_mr,abs_angle
0,0.204545,0.390411,0.698052,0.392694,0.405844,0.618721,0.194805,0.712329,0.733766,0.705479,...,0.249515,0.615908,0.693416,0.314818,0.446237,0.339205,0.350364,0.539005,0.539049,0.061566
1,0.179724,0.397351,0.589862,0.403974,0.290323,0.619205,0.202765,0.725166,0.635945,0.728477,...,0.171500,0.563722,0.648461,0.327759,0.453958,0.362484,0.377599,0.433192,0.433204,0.134023
2,0.195946,0.404878,0.662162,0.404878,0.378378,0.614634,0.202703,0.731707,0.682432,0.731707,...,0.239078,0.586077,0.664537,0.327457,0.453156,0.325814,0.344595,0.479730,0.479730,0.174672
3,0.266055,0.373239,0.715596,0.373239,0.440367,0.549296,0.238532,0.690141,0.724771,0.704225,...,0.272772,0.565660,0.629561,0.331113,0.431290,0.323865,0.348745,0.486442,0.486585,0.121842
4,0.244565,0.389764,0.695652,0.393701,0.413043,0.578740,0.217391,0.696850,0.706522,0.712598,...,0.254682,0.563583,0.641880,0.319083,0.440352,0.322564,0.346806,0.489384,0.489613,0.012195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,0.196911,0.401146,0.637066,0.383954,0.339768,0.616046,0.262548,0.759312,0.610039,0.753582,...,0.207921,0.543033,0.629450,0.370614,0.498802,0.303253,0.327708,0.347538,0.347576,0.540420
2301,0.255892,0.386256,0.767677,0.398104,0.484848,0.649289,0.282828,0.772512,0.686869,0.779621,...,0.267332,0.583504,0.705787,0.389981,0.548077,0.240413,0.274054,0.404103,0.404167,0.552494
2302,0.240838,0.400372,0.725131,0.396648,0.476440,0.616387,0.314136,0.780261,0.675393,0.780261,...,0.281800,0.577194,0.688497,0.386824,0.541556,0.257753,0.304386,0.361257,0.361257,0.162489
2303,0.198381,0.405836,0.676113,0.413793,0.331984,0.631300,0.214575,0.801061,0.574899,0.814324,...,0.284469,0.555543,0.728351,0.413121,0.619658,0.304147,0.370196,0.360568,0.360892,0.456213


In [11]:
# Mencatat waktu berakhirnya keseluruhan program ekstraksi informasi wajah pada training images.
global_end_time = time.time()

# Menampilkan waktu eksekusi dari keseluruhan program ekstraksi informasi wajah pada training images.
total_execution_time = datetime.timedelta(seconds = global_end_time - global_start_time)
print("total execution time: %s" % (total_execution_time))

total execution time: 0:28:25.535156
